In [1]:
import pandas as pd
import numpy as np
import os
import glob
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import wfdb
import statsmodels.api as sm
import matplotlib.image as mpimg
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.backends.backend_tkagg import NavigationToolbar2Tk
from matplotlib.figure import Figure
from IPython.display import display, HTML
import csv
from collections import defaultdict
import matplotlib.ticker as ticker
import pickle
import matplotlib as mpl
from sklearn.metrics import r2_score
import warnings
import tensorflow.keras as ks
from RDFunctions import *

In [2]:
def plotGraph(f,canvas,serial_id,
              OUTLIERSD,LOWESSWINDOW,REGRESSIONWINDOW, \
              SCATTER, LOWESS, REGRESSION, \
              disprr, dispspo2, \
              LEFTEND, RIGHTEND, \
              HEADERLINE,ANNOTATE,dfOuranno, \
              SPO2_THRESH,RR_THRESH,SPO2_SLOPE_THRESH,RR_SLOPE_THRESH, \
              LR_model, LGB_model, MLP_model, SVM_model, CNN_model, LSTM_model):
    
    record_name = dfOuranno.loc[serial_id,'RecordNum'] 
    patient_id = dfOuranno.loc[serial_id,'PatientId']
    flname = './FinalRecords/'+str(record_name)+'n.hea'
    recname = './FinalRecords/'+str(record_name)+'n'       
    annodataOur = dfOuranno[dfOuranno['RecordNum']==int(record_name)]
    ANNOTATE.delete(1.0, 'end')
    ANNOTATE.insert('end',str(annodataOur))
    
    [samples,R,S,H,firstline] = getIndexOfRRSpO2HR(flname)
    HEADERLINE.set(firstline)
    rec =  wfdb.io.rdrecord(str(recname))
    xrr = rec.p_signal[:,R]
    #print(xrr)
    xspo2 = rec.p_signal[:,S]
    xhr = rec.p_signal[:,H]
    TOTAL_LEN = len(xrr)
    t = np.arange(0,TOTAL_LEN,1)
    
    xrr = xrr[int(LEFTEND*TOTAL_LEN/100.0):int(RIGHTEND*TOTAL_LEN/100.0)]
    xspo2 = xspo2[int(LEFTEND*TOTAL_LEN/100.0):int(RIGHTEND*TOTAL_LEN/100.0)]
    xhr = xhr[int(LEFTEND*TOTAL_LEN/100.0):int(RIGHTEND*TOTAL_LEN/100.0)]
    t = t[int(LEFTEND*TOTAL_LEN/100.0):int(RIGHTEND*TOTAL_LEN/100.0)]
    
    [xrrnew,trrnew] = outlierRejector(xrr,t,OUTLIERSD,default=15.0)
    [xspo2new,tspo2new] = outlierRejector(xspo2,t,OUTLIERSD, default=98.0)
    [xhrnew,thrnew] = outlierRejector(xhr,t,OUTLIERSD)
    
    zrrnew = applyLowess(xrrnew,trrnew,LOWESSWINDOW*60)
    zspo2new = applyLowess(xspo2new,tspo2new,LOWESSWINDOW*60)
    zhrnew = applyLowess(xhrnew,thrnew,LOWESSWINDOW*60)
    

    mRR,cRR,rRR = fitRegressionLines(zrrnew[:,0]/60,(zrrnew[:,1]),int(REGRESSIONWINDOW*60))
    mSPO2,cSPO2,rSPO2 = fitRegressionLines(zspo2new[:,0]/60,(zspo2new[:,1]),int(REGRESSIONWINDOW*60))
    
   
    plt = f.add_subplot(111)
  
    plt.clear()
    font = {'size': 15, 'weight':'bold'}
    mpl.rc('font', **font)
    plt.set_title("RR/SpO2")
    plt.set_ylim(-85,105)
    
    if(SCATTER==1):
        if(disprr == 1):
            plt.scatter(t/60.0,xrr,c='hotpink',marker='.',alpha='0.1')
            #plt.scatter(t/60.0,xhr,c='magenta',marker='.',alpha='0.1')
        if(dispspo2 == 1):
            plt.scatter(t/60.0,xspo2,c='cornflowerblue',marker='.',alpha='0.05')
    
    if(LOWESS==1):
        if(disprr == 1):
            plt.plot(zrrnew[:,0]/60.0,(zrrnew[:,1]),'--r',label='RR',linewidth=3)
            #plt.plot(zhrnew[:,0]/60.0,(zhrnew[:,1]),'--m',label='HR',linewidth=3)
        if(dispspo2 == 1):
            plt.plot(zspo2new[:,0]/60.0,(zspo2new[:,1]),'--b',label='SpO2',linewidth=3)
    
    tnew = zrrnew[:,0]/60.0
    points = int(REGRESSIONWINDOW*60)
    
    if(REGRESSION==1): 
        if(disprr == 1):
            for i in range(len(mRR)):
                fit_fn = np.poly1d([mRR[i],cRR[i]])
                plt.plot(tnew[i*points:(i+1)*points], fit_fn(tnew[i*points:(i+1)*points]),'r',linewidth=3.0)
        if(dispspo2 == 1):
            for i in range(len(mSPO2)):
                fit_fn = np.poly1d([mSPO2[i],cSPO2[i]])
                plt.plot(tnew[i*points:(i+1)*points], fit_fn(tnew[i*points:(i+1)*points]),'b',linewidth=3.0)
    
    #print(tnew)
    #print(xrr)
    #print(t)
    #print(points)
    seginred = 0
    if(REGRESSION==1):
        T = 1.0
        col = '#00ff00'
        plt.hlines(y=45.0, xmin=tnew[0], xmax=tnew[-1], linewidths=20.0, colors=col)
        plt.hlines(y=55.0, xmin=tnew[0], xmax=tnew[-1], linewidths=20.0, colors=col)
        plt.hlines(y=65.0, xmin=tnew[0], xmax=tnew[-1], linewidths=20.0, colors=col)
        plt.text(0, 41, 'Trend Bar', fontsize=10)
        plt.text(0, 51, 'RR Bar', fontsize=10)
        plt.text(0, 61, 'SpO2 Bar', fontsize=10)
        for i in range(len(mRR)):
            START = i*points
            END = START + points
            if (END >= len(tnew)):
                END = -1
                
            if(np.mean(zrrnew[START:END,1]) > RR_THRESH):
                col = '#ffbbbb'
                plt.hlines(y=55.0, xmin=tnew[START], xmax=tnew[END], linewidths=20.0, colors=col)
            
            if(np.mean(zspo2new[START:END,1]) < SPO2_THRESH):
                col = '#ffbbbb'
                plt.hlines(y=65.0, xmin=tnew[START], xmax=tnew[END], linewidths=20.0, colors=col)

            if(mRR[i] > RR_SLOPE_THRESH and mSPO2[i] >= -SPO2_SLOPE_THRESH and mSPO2[i] <= SPO2_SLOPE_THRESH):
                col = '#ffbbbb'
                plt.axvline(tnew[START],color='k',linestyle='--')
                plt.axvline(tnew[END],color='k',linestyle='--')
                plt.hlines(y=45.0, xmin=tnew[START], xmax=tnew[END], linewidths=20.0, colors=col)
                seginred = seginred + 1
            elif(mRR[i] > RR_SLOPE_THRESH and mSPO2[i] < -SPO2_SLOPE_THRESH):
                col = '#ff7777'
                plt.axvline(tnew[START],color='k',linestyle='--')
                plt.axvline(tnew[END],color='k',linestyle='--')
                plt.hlines(y=45.0, xmin=tnew[START], xmax=tnew[END], linewidths=20.0, colors=col)
                seginred = seginred + 1
            elif(mRR[i] < -RR_SLOPE_THRESH and mSPO2[i] < -SPO2_SLOPE_THRESH):
                col = '#ff0000'
                plt.axvline(tnew[START],color='k',linestyle='--')
                plt.axvline(tnew[END],color='k',linestyle='--')
                plt.hlines(y=45.0, xmin=tnew[START], xmax=tnew[END], linewidths=20.0, colors=col)
                seginred = seginred + 1
        
        rr_loess = zrrnew[:,1]
        spo2_loess = zspo2new[:,1]
        
        bar_x = []
        bar_h_lr = []
        bar_h_cnn = []
        bar_h_svm = []
        bar_h_mlp = []
        bar_h_lgb = []
        bar_h_lstm = []
        seg = np.zeros((1,1440,2),dtype=np.float64)
        for ii in range(24*60,TOTAL_LEN,60):
            feature_array = getLongFeatures(rr_loess[(ii-24*60):ii],spo2_loess[(ii-24*60):ii])
            seg[0,:,0] = (rr_loess[(ii-24*60):ii] - 25)/10
            seg[0,:,1] = (spo2_loess[(ii-24*60):ii] - 93)/10
            feature_array = np.array(feature_array[0:12],dtype=np.float32)
            
            lr_predict = LR_model.predict_proba(feature_array.reshape(1,-1))[0][1]
            lgb_predict = LGB_model.predict_proba(feature_array.reshape(1,-1))[0][1]
            mlp_predict = MLP_model.predict_proba(feature_array.reshape(1,-1))[0][1]
            svm_predict = SVM_model.predict_proba(feature_array.reshape(1,-1))[0][1]
            cnn_predict = CNN_model.predict(seg)[0,0]
            lstm_predict = LSTM_model.predict(seg)[0,0]
            
            bar_x.append(float(ii)/60.0)
            
            bar_h_lr.append(10*lr_predict)
            bar_h_lgb.append(10*lgb_predict)
            bar_h_mlp.append(10*mlp_predict)
            bar_h_svm.append(10*svm_predict)
            bar_h_cnn.append(10*cnn_predict)
            bar_h_lstm.append(10*lstm_predict)
            
        plt.hlines(y=0.0, xmin=tnew[0], xmax=tnew[-1], linewidths=1.0, colors='magenta')
        plt.bar(bar_x,bar_h_lr,bottom=-10.0, color='magenta')
        plt.hlines(y=-10.0, xmin=tnew[0], xmax=tnew[-1], linewidths=1.0, colors='magenta')
        plt.text(0, -5, 'SI by LR', fontsize=20, color='magenta', fontweight='bold')
        
        plt.hlines(y=-15.0, xmin=tnew[0], xmax=tnew[-1], linewidths=1.0, colors='#004C99')
        plt.bar(bar_x,bar_h_cnn,bottom=-25.0, color='#004C99')
        plt.hlines(y=-25.0, xmin=tnew[0], xmax=tnew[-1], linewidths=1.0, colors='#004C99')
        plt.text(0, -20, 'SI by CNN',fontsize=20, color='#004C99', fontweight='bold')
        
        plt.hlines(y=-30.0, xmin=tnew[0], xmax=tnew[-1], linewidths=1.0, colors='#009999')
        plt.bar(bar_x,bar_h_svm,bottom=-40.0, color='#009999')
        plt.hlines(y=-40.0, xmin=tnew[0], xmax=tnew[-1], linewidths=1.0, colors='#009999')
        plt.text(0, -35, 'SI by SVM', fontsize=20, color='#009999', fontweight='bold')
        
        plt.hlines(y=-45.0, xmin=tnew[0], xmax=tnew[-1], linewidths=1.0, colors='#999900')
        plt.bar(bar_x,bar_h_mlp,bottom=-55.0, color='#999900')
        plt.hlines(y=-55.0, xmin=tnew[0], xmax=tnew[-1], linewidths=1.0, colors='#999900')
        plt.text(0, -50, 'SI by MLP', fontsize=20, color='#999900', fontweight='bold')
        
        plt.hlines(y=-60.0, xmin=tnew[0], xmax=tnew[-1], linewidths=1.0, colors='#999900')
        plt.bar(bar_x,bar_h_lgb,bottom=-70.0, color='#999900')
        plt.hlines(y=-70.0, xmin=tnew[0], xmax=tnew[-1], linewidths=1.0, colors='#999900')
        plt.text(0, -65, 'SI by LGB', fontsize=20, color='#999900', fontweight='bold')
        
        plt.hlines(y=-75.0, xmin=tnew[0], xmax=tnew[-1], linewidths=1.0, colors='#004C99')
        plt.bar(bar_x,bar_h_lstm,bottom=-85.0, color='#004C99')
        plt.hlines(y=-85.0, xmin=tnew[0], xmax=tnew[-1], linewidths=1.0, colors='#004C99')
        plt.text(0, -80, 'SI by LSTM',fontsize=20, color='#004C99', fontweight='bold')
        
        
        filtSpO2Alarms = getFilteredLowAlarm(zspo2new[:,1],tnew,92.0)
        filtRRAlarms = getFilteredHighAlarm(zrrnew[:,1],tnew,24.0)
        for j in range(len(filtRRAlarms)):
            plt.axvline(x=filtRRAlarms[j], linewidth=2.0, color='red')
        for j in range(len(filtSpO2Alarms)):
            plt.axvline(x=filtSpO2Alarms[j], linewidth=2.0, color='blue')
    
    
   
    mpl.rc('font', **font)
    plt.yaxis.set_major_locator(ticker.MultipleLocator(5))
    plt.grid(color='gray', linestyle='-', linewidth=0.5, which='both')
    canvas.draw()

In [3]:
warnings.filterwarnings("ignore",category=DeprecationWarning)
# Create a canvas
top = tk.Tk()

# load logistic regression model from disk
LR_model = pickle.load(open('logistic_model.sav', 'rb'))

#load LGB model from disk
LGB_model = pickle.load(open('lgb_model.sav', 'rb'))

#load MLP regression model from disk
MLP_model = pickle.load(open('mlp_model.sav', 'rb'))

# load SVM Model from disk
SVM_model = pickle.load(open('svm_model.sav', 'rb'))

#load 1D CCN model from disk
CNN_model = ks.models.load_model('QuantCNN.h5')

#load LSTM model from disk
LSTM_model = ks.models.load_model('QuantLSTM.h5')


firstFrame = tk.Frame(top)
firstFrame.pack(side='top')
serial_id = tk.IntVar(top)
serial_id.set(0)


annofile = r'RecordsAnno3.csv'
dfOuranno = pd.read_csv(annofile, encoding='iso-8859-1')    
    

OMrecordserialid = tk.Entry(firstFrame,textvariable=serial_id,bd=5)
OMrecordserialid.pack(side='left', padx=20)


disprr = tk.IntVar(top)
disprr.set(1)
CBdisprr = tk.Checkbutton(firstFrame, text = "RR", variable = disprr, onvalue = 1, offvalue = 0, height=5, width = 5)
CBdisprr.pack(side='left', padx=5)

dispspo2 = tk.IntVar(top)
dispspo2.set(1)
CBdispspo2 = tk.Checkbutton(firstFrame, text = "SpO2", variable = dispspo2, onvalue = 1, offvalue = 0, height=5, width = 7)
CBdispspo2.pack(side='left', padx=5)

SCATTER = tk.IntVar(top)
SCATTER.set(1)
CBSCATTER = tk.Checkbutton(firstFrame, text = "Actual", variable = SCATTER, onvalue = 1, offvalue = 0, height=5, width = 7)
CBSCATTER.pack(side='left', pady=20)

LOWESS = tk.IntVar(top)
LOWESS.set(1)
CBLOWESS = tk.Checkbutton(firstFrame, text = "Lowess", variable = LOWESS, onvalue = 1, offvalue = 0, height=5, width = 7)
CBLOWESS.pack(side='left', padx=5)

REGRESSION = tk.IntVar(top)
REGRESSION.set(1)
CBREGRESSION = tk.Checkbutton(firstFrame, text = "Linear", variable = REGRESSION, onvalue = 1, offvalue = 0, height=5, width = 7)
CBREGRESSION.pack(side='left', padx=5)

HEADERLINE = tk.StringVar(top)
HEADERLINE.set("Header First Line")
headerLabel = tk.Label(firstFrame, textvariable = HEADERLINE)
headerLabel.config(height=2, width=100)       
headerLabel.pack(side='top')

# create a Text widget
ANNOTATE = tk.Text(firstFrame, borderwidth=3, height=4, width=80)
ANNOTATE.config(font=("consolas", 12), undo=True, wrap='none')
ANNOTATE.pack(side='top',fill="x", expand=False)


# create a Scrollbar and associate it with txt
scrollbar = tk.Scrollbar(firstFrame, command=ANNOTATE.xview, orient='horizontal')
#scrollb.grid(row=0, column=1, sticky='nsew')
ANNOTATE['xscrollcommand'] = scrollbar.set
scrollbar.pack(side='top',fill="x", expand=False)
#-------------------------------------------------------------------------------
secondFrame = tk.Frame(top)
secondFrame.pack(side='top')

#f = Figure(figsize=(16.2,3.5)) # for laptop version
f = Figure(figsize=(21,10))  # for desktop version
#myplot = f.add_subplot(111)
#myplot.plot([1,2,3,4,5,6,7,8],[5,6,1,3,8,9,3,5])
canvas = FigureCanvasTkAgg(f,secondFrame)
canvas.get_tk_widget().pack(side=tk.TOP)
toolbar = NavigationToolbar2Tk(canvas, secondFrame)
toolbar.update()
canvas._tkcanvas.pack(side=tk.TOP, fill=tk.BOTH, expand=1)

LEFTEND = tk.DoubleVar(top)
LEFTEND.set(0.0)
# laptop version
#Sdataleft = tk.Scale(secondFrame, variable = LEFTEND, from_=0.0, to=100.0, orient='horizontal',resolution=0.1, length='10i')

# desktop version
Sdataleft = tk.Scale(secondFrame, variable = LEFTEND, from_=0.0, to=100.0, orient='horizontal',resolution=0.1, length='18i')
labelD = tk.Label(secondFrame, text = "Left End Data (%)")
Sdataleft.pack(side='top')
labelD.pack(side='top')

RIGHTEND = tk.DoubleVar(top)
RIGHTEND.set(100.0)
# for laptop version
#Sdataright = tk.Scale(secondFrame, variable = RIGHTEND, from_=0.0, to=100.0, orient='horizontal',resolution=0.1, length='10i')

# for desktop version
Sdataright = tk.Scale(secondFrame, variable = RIGHTEND, from_=0.0, to=100.0, orient='horizontal',resolution=0.1, length='18i')

labelE = tk.Label(secondFrame, text = "Right End Data (%)")
Sdataright.pack(side='top')
labelE.pack(side='top')





#--------------------------------------------------------------------------------

thirdFrame = tk.Frame(top)
thirdFrame.pack(side='right')

OUTLIERSD = tk.DoubleVar(top)
OUTLIERSD.set(2.5)
Soutliersd = tk.Scale(thirdFrame, variable = OUTLIERSD, from_=0.1, to=5.0, orient='horizontal',resolution=0.1)
labelA = tk.Label(thirdFrame, text = "Outlier Rejection (SD)")
Soutliersd.pack(side='top')
labelA.pack(side='top')


REGRESSIONWINDOW = tk.DoubleVar(top)
REGRESSIONWINDOW.set(1.0)
Sregressionwindow = tk.Scale(thirdFrame, variable = REGRESSIONWINDOW, from_=0.5, to=12.0, orient='horizontal',resolution=0.1)
labelC = tk.Label(thirdFrame, text = "Linear Regression Window (hrs)")
Sregressionwindow.pack(side='top')
labelC.pack(side='top')

#-------------------------------------------------------------------------------

fourthFrame = tk.Frame(top)
fourthFrame.pack(side='left')

SPO2_THRESH = tk.DoubleVar(top)
SPO2_THRESH.set(93.0)
scale_SPO2_THRESH = tk.Scale(fourthFrame,label='SpO2 Threshold',variable = SPO2_THRESH, from_=80.0, to=98.0, 
                             orient='horizontal',resolution=0.1, length='2i')
scale_SPO2_THRESH.pack(side='left',pady=15)

RR_THRESH = tk.DoubleVar(top)
RR_THRESH.set(25.0)
scale_RR_THRESH = tk.Scale(fourthFrame,label='RR Threshold',variable = RR_THRESH, from_=15.0, to=35.0, 
                             orient='horizontal',resolution=0.1, length='2i')
scale_RR_THRESH.pack(side='left',pady=15)


SPO2_SLOPE_THRESH = tk.DoubleVar(top)
SPO2_SLOPE_THRESH.set(1.0)
scale_SPO2_SLOPE_THRESH = tk.Scale(fourthFrame,label='SpO2 Slope Thresh',variable = SPO2_SLOPE_THRESH, from_=1.0, to=3.0, 
                                   orient='horizontal',resolution=0.1, length='2i')
scale_SPO2_SLOPE_THRESH.pack(side='left',pady=15)

RR_SLOPE_THRESH = tk.DoubleVar(top)
RR_SLOPE_THRESH.set(1.0)
scale_RR_SLOPE_THRESH = tk.Scale(fourthFrame,label='RR Slope Thresh',variable = RR_SLOPE_THRESH, from_=1.0, to=3.0, 
                                 orient='horizontal',resolution=0.1, length='2i')
scale_RR_SLOPE_THRESH.pack(side='left',pady=15)



Bplot = tk.Button(fourthFrame, text ="Plot Graph", \
                  command = lambda: plotGraph(f,canvas,int(serial_id.get()), \
                                              OUTLIERSD.get(),3.0*REGRESSIONWINDOW.get(),REGRESSIONWINDOW.get(), \
                                             SCATTER.get(), LOWESS.get(), REGRESSION.get(), \
                                             disprr.get(), dispspo2.get(), \
                                             LEFTEND.get(), RIGHTEND.get(), \
                                              HEADERLINE, ANNOTATE, dfOuranno, \
                                              SPO2_THRESH.get(),RR_THRESH.get(), \
                                              SPO2_SLOPE_THRESH.get(),RR_SLOPE_THRESH.get(), \
                                             LR_model, LGB_model, MLP_model, SVM_model, CNN_model, LSTM_model))

Bplot.pack(side='left',pady=20)
top.mainloop()

